# CART: Classification and Regression Trees

# Introduction and Dataset

## Background

**Insert background** 

## Tutorial goals

**Goal 1: Develope a random forest classification model for dominant tree species**

**Goal 2: Train RF for tree species classification using five-fold cross validation**

**Goal 3: Experiment with different hyperparameters to limit overfitting**

**Goal 4: Investigate goals 1 - 3 for aboveground biomass regression using RF**

-----

## Data

Please refer to the README on the main GitHub page for a detailed description of each file.